## Объекты

In [21]:
from array import array
import math

class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>

* 1 ```typecode``` - это атрибут класса, которым мы воспользуемся, когда будем преобразовывать экземпляры ```Vector2d``` в последовательности байтов и наоборот
* 2 Преобразование ```x``` и ```y``` в тип ```float``` в методе ```__init__``` позволяет на ранней стадии обнаружить оишбки, это полезно в случае, когда конструктор ```Vector2d``` вызывается с неподходящими аргументами
* 3 Наличие метода ```__iter__``` делает ```Vector2d``` итерируемым; именно благодаря ему работает распаковка (например: ```x, y = my_vector```). Мы реализуем его просто с помощью генераторного выражения, которое отдает компоненты поочередно.
* 4 Метод ```__repr__``` строит строку, интеполируя компоненты с помощью синаксиса ```{!r}``` для получения их представления, возвращаемого функции ```repr```; поскольку ```Vector2d``` - итерируемый объект, ```*self``` поставляет компоненты ```x``` и ```y``` функции ```format```
* 5 Из итерируемого объекта ```Vector2d``` легко построить кортеж для отображения в виде упорядоченной пары
* 6 Для генерации объекта типа ```bytes``` мы преобразуем ```typecode``` в ```bytes``` и конкатенируем
* 7 ... с объектом ```bytes```, полученным преобразованием массива, который построен путем обхода экземпляра
* 8 Для быстрого сравнения всех компонентов мы строим кортежи из операндов. Это работает, когда операнды являются экземплярами класса ```Vector2d```, но не без проблем
* 9 Модулем вектора называется длина гипотенузы прямоугольного треугольника с катетами ```x``` и ```y```
* 10 Метод ```__bool__``` вызывает ```abs(self)``` для вычисления модуля, а затем преобразует полученное значение в тип ```bool```, так что 0.0 преобразуется в ```False```, а любое число, отличное от нуля, - в ```True```

In [22]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y) # 1

3.0 4.0


In [23]:
x, y = v1 # 2
x, y

(3.0, 4.0)

In [24]:
v1 # 3

Vector2d(3.0, 4.0)

In [25]:
v1_clone = eval(repr(v1)) # 4
v1 == v1_clone # 5

True

In [26]:
print(v1) # 6

(3.0, 4.0)


In [27]:
ocerts = bytes(v1) # 7
ocerts

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [28]:
abs(v1) # 8

5.0

In [29]:
bool(v1), bool(Vector2d(0, 0)) #9

(True, False)

* 1 К компонентам ```Vector2d``` можно обращаться напрямую, как к атрибутам *(методом чтения нет)*
* 2 Объект ```Vector2d``` можно распоковать в кортеж переменных
* 3 ```repr``` для объекта имитирует исходный конструированияэкземпляра
* 4 Использование ```eval``` показывает, что результат ```repr``` для ```Vector2d``` - точное представление вызова конструктора
* 5 ```Vector2d``` поддерживает сравнение с помощью ```==```; это полезно для тестирования
* 6 ```print``` вызывает функцию ```str```, которая для ```Vector2d``` порождает упорядоченную пару
* 7 ```bytes``` используется методом ```__bytes__``` для получения двоичного представления
* 8 ```abs``` вызывает метод ```__abs__```, чтобы вернуть модуль вектора
* 9 ```Vector2d``` пользуется методом ```__bool__```, чтобы вернуть ```False``` для объекта ```Vector2d``` нулевой длины, и ```True``` в противном случае

## Альтернативный конструктор

In [30]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # <4>
        return cls(*memv)  # <5>

* 1 Метод класса снабжен декоратором ```classmethod```
* 2 Аргумент ```self``` отсутвует; вместо него в аргументе ```cls``` прередается сам класс
* 3 Читаем ```typecode``` из первого байта
* 4 Создаем объект ```memoryview``` из двоичной последовательности октетов и приводим его к типу ```typecode```
* 5 Распаковываем ```memoryview```, получившийся в результате приведения типа,и получаем пару аргументов, необходимых конструктору

## Декораторы ```classmethod``` и ```staticmethod```

In [31]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args # 1

    @staticmethod
    def statmeth(*args):
        return args # 2

Demo.klassmeth() # 3

(__main__.Demo,)

In [32]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [33]:
Demo.statmeth() # 4

()

In [34]:
Demo.statmeth('spam')

('spam',)

* 1 ```klassmeth``` просто возвращает все позиционные аргументы
* 2 ```statmeth```делает тоже самое
* 3 Вне зависимости от способа вызова ```Demo.klassmeth``` получает класс ```Demo``` в качестве первого аргумента
* 4 ```Demo.statmeth``` ведет себя, как обычная функция

## Форматирование при выводе

In [35]:
brl = 1/5.46 # Курс Бразильского реала к доллару
brl

0.18315018315018314

In [36]:
format(brl, '0.4f') # 1

'0.1832'

In [37]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl) # 2

'1 BRL = 0.18 USD'

* 1 Спецификатор формата ``` ```
* 2 Спецификатор формата  ```0.2f```. Подстройка ```rate``` в поле подстановки называется именем поля. Она не свзяна со спецификатором формата, а определяет, какой аргумент метода ```.foramat()``` попадает в это поле подстановки.

Например ```{0.mass:5.3e}``` мы видим две соверешенно разных нотации. Часть ```0.mass``` слева от двоеточия - это имя поля подстановки ```field.name```, а часть ```5.3e``` после двоеточия - спецификатор формата. Нотация, применяемая в спецификаторе формата, называется также **миниязыком спецификации формата** 

In [38]:
format(42, 'b') # Двоичная система

'101010'

In [39]:
format(42, 'x') # 16-ричная система

'2a'

In [40]:
format(2/3, '.1%')

'66.7%'

In [41]:
from datetime import datetime

now = datetime.now()
format(now, '%H:%M')

'10:30'

In [42]:
'Сейчас {:%I:%M %p}'.format(now)

'Сейчас 10:30 AM'

Если в классе не рализован метод ```__format__```, то используется метод, унаследованный от ```object```, который возвращает значение ```str(my_object)```. Поскольку в классе ```Vector2d``` есть метод ```__str__```, это работает следующим образом:

In [43]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [44]:
v1

Vector2d(3.0, 4.0)

Но если передать спецификатор формата, то ```object.__format__``` возбудит исключение ```TypeError``` 

In [45]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [49]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):  # <1>
            fmt_spec = fmt_spec[:-1]  # <2>
            coords = (abs(self), self.angle())  # <3>
            outer_fmt = '<{}, {}>'  # <4>
        else:
            coords = self  # <5>
            outer_fmt = '({}, {})'  # <6>
        components = (format(c, fmt_spec) for c in coords)  # <7>
        return outer_fmt.format(*components)  # <8>

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

* 1 Формат заканчивается буквой ```'p'```: полярные координаты
* 2 Удаляем суффикс ```'p'``` из ```fmt_spec```
* 3 Строим кортеж полярных координат: ```(magnitude, angle)```
* 4 Конфигурируем внешний формат, используя угловые скобки
* 5 Иначе используем компоненты ```x, y``` вектора ```self``` для представления в прямоугольных координатах
* 6 Конфигурируем внешний формат, используя круглые скобки
* 7 Порождаем итерируемый объект, компонентами которого являются отформатированные строки
* 8 Подставляем строки во внешний формат

In [51]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [52]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [54]:
format(Vector2d(1, 1), '0.4p')

'<1.414, 0.7854>'

In [50]:
a = Vector2d(1, 1)
format(a, '.3f')

'(1.000, 1.000)'

## Хэшируемый класс

До сих поря экземпляры класса ```Vector2d``` были не хэшируемыми, поэтому мы не могли поместить их в множество

In [55]:
v1 = Vector2d(3 ,4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

Что бы класс ```Vector2d``` был хэшируемым, мы должны реализовать метод ```__hash__```(необходим еще метод ```__eq__```, но он у нас уже есть). Также нужно, чтобы векторы были неизменяемыми


In [56]:
v1.x = 7

In [57]:
v1

Vector2d(7, 4.0)

Мы добъемся этого, сделав компоненты ```x``` и ```y``` свойствами, доступными только для чтения

In [58]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)  # <1>
        self.__y = float(y)

    @property  # <2>
    def x(self):  # <3>
        return self.__x  # <4>

    @property  # <5>
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y)) 

    # remaining methods follow (omitted in book listing)
# END VECTOR2D_V3_PROP

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

# BEGIN VECTOR_V3_HASH
    def __hash__(self):
        return hash(self.x) ^ hash(self.y) # 6
# END VECTOR_V3_HASH

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

* 1 Используем ровнодва начальных подчерка(и нуль или один конечный), чтобы сделать атрибут закрытым
* 2 Декоратор ```@property``` помечает метод чтения свойства
* 3 Просто возвращаем ```self.__x```
* 4 Повторяем то же самое для свойства ```y```
* 5 Все методы, которые просто читают компоненты ```self.x``` и ```self.y```, не изменяются, только теперь ```x``` и ```y```, означает чтение открытых свойств, а не закрытых атрибутов.
* 6 Метод ```__hash__``` должен возвращать ```int```, и в идеале учитывать хэши объектов-атрибутов, которые используются в методе ```__eq__```, потому что у объектов должны быть одинаковые хэши. Также рекомендуется объединять хэши компонентов, с помощью поразрядного опретора **ИСКЛЮЧАЮЩЕЕ ИЛИ** (```^```)

In [59]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [60]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## Закрытые и "защищенные" атрибуты в Python

Рассмотрим такую ситуацию: кто-то написал класс ```Dog```, в котором используется внутренний атрибут экземпляра ```mood```, который автор не захотел раскрывать клиентам. Нам нужно написать подкасс ```Dog - Beagle```. Если мы создадим свой атрибут экземпляра ```mood```, не подозревая о конфликте имен, то затрем атрибут ```mood```, испольщуемый в методах, унаследованных от ```Dog```. Чтобы предотвратить это, мы можем назвать атрибут ```__mood``` (с двумя начальными прочерками и, возможно, одним - не более - конечным прочекрком). Тогда Python сохранить имя в словаре экземпляра ```__dict__```, добавив в начало один подчерк и имя класса, т.е. в классе ```Dog``` атрибут ```__mood``` будет называться ```_Dog__mood```, а в классе ```Beagle - _Beagle__mood```. Эта особенность языка называется *ДЕКОРИРОВАНИЕ ИМЕН(name mangling)* 

In [62]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [63]:
v1

Vector2d(3.0, 4.0)

In [64]:
v1._Vector2d__x = -7

In [65]:
v1

Vector2d(-7, 4.0)

## Экономия памяти с помощью атрибута класса ```__slots__```

По умолчанию Python хранит атрибуты экземпляра в словаре ```__dict__```, принадлежащему самому экземпляру. Со словарями сопрежены значитаельные накладные расходы из-за того что, для обеспечения быстрого доступа используется хэш-таблица. Если имеются миллионы экземпляров, а колличество атрибутов у каждого мало, то атрибут класса ```__slots__```позволит сэкономить очень много памяти за счет того, что интерпетатору разрешено хранить атрибуты экзэмпляра не в словаре, а в кортеже.

Атрибут ```__slots__```, унаследованный от суперкласса, не оказывает никакого влияния. Python принимает в расчет только атрибуты ```__slots__```, определенные в самом классе.

In [68]:
class Vector2d:
    __slots__ = ('__x', '__y') # 1

    typecode = 'd'

    # methods follow (omitted in book listing)
# END VECTOR2D_V3_SLOTS

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

* 1 Определяя атрибут ```__slots__```, мы говорим интерпретатору "Это все атрибуты в данном классе". Тогда все переменные помещаются в кортежную структуру, что позволяет избежать накладных расходов хранения в ```__dict__```

Если в классе определен атрибут ```__slots__```, то запрещается включать в его экземпляры какие-либот атрибуты, кроме перечисленных в ```__slots__```. Но это побочный эффект, а не причина существования ```__slots__```. Считается дурным тоном использовать ```__slots__``` только для того, чтобы не дать пользователям класса создавать новые атрибуты в его экземплярах. **Атрибут ```__slots__``` предназначен для оптимизации**, а не для связвания рук программистам.

Однако же возможно и "память сэкономить, и косточкой не подавится": если добавить имя ```__dict__``` в список ```__slots__```, то все атрибуты, перечисленные в ```__dict__```, будут хранится в кортеже, принадлежащим экземпляру, но при этом разрешено динамически создавать новые атрибуты, которые хранятся в словаре ```__dict__```, как обычно. Разумеется, помещение ```__dict__``` в атрибут ```__slots__``` может свести все преимущества последнего, но это уже зависит от колличества статических и динамических атрибутов, и того как они используются. 

Существует еще один специальный атрибут экземпляра, который имеет смысл сохранить: ```__weakref__``` необходим, чтобы объект поддерживал слабые ссылки. Если в классе определен атрибут ```__slots__```, а вам нужно чтобы его экземпляры могли быть объектами слабых сылок, то ```__weakref``` необходимо явно включить в список имен атрибутов ```__slots__```.

* Не забывайте заново объявлять ```__slots__``` в каждом подклассе, потому что унаследованный атрибут интепретатор игнорирует
* Экземпляры класса могут иметь атрибуты, явно перечисленные в ```__slots__```, если не включено также имя ```__dict__```
* Экземпляры класса не могут быть объектами слабых ссылок, если не включить в ```__slots__``` имя ```__weakref__```

#### Результаты выполнения тестов

```
$ time python3 mem_test.py Vector2d.py 
Selected Vector2d type: Vector2d.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:          9,300
  Final RAM usage:      1,655,928
python3 mem_test.py Vector2d.py  7,86s user 0,35s system 99% cpu 8,246 total
```

```
$ time python3 mem_test.py Vector2d_slots.py 
Selected Vector2d type: Vector2d_slots.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:          9,428
  Final RAM usage:        558,136
python3 mem_test.py Vector2d_slots.py  5,82s user 0,10s system 99% cpu 5,933 total
```

## Переопределение атрибутов класса

In [70]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [72]:
len(dumpd) # 1

17

In [75]:
v1.typecode = 'f' # 2
dumpd = bytes(v1)
dumpd

AttributeError: 'Vector2d' object attribute 'typecode' is read-only

* 1 Подразумевается по умолчанию представление ```bytes``` имеет длину в 17 байтов
* 2 Присваиваем ```typecode``` значение ```'f'``` в экземпляре ```v1``` 

In [76]:
class ShortVector2d(Vector2d): # 1
    typecode = 'f'

In [77]:
sv = ShortVector2d(1/11, 1/27) # 2 
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [78]:
len(bytes(sv)) # 3

9

* 1 Создаем ```ShortVector2d``` как подкласс ```Vector2d``` только для того, чтобы переопределить атрибут класса ```typecode```
* 2 Создаем экземпляр ```ShortVector2d``` - объект ```sv```
* 3 Проверяем, что экспортировано 9 байтов, а не 17, как раньше

Этот пример также объясняет, почему значение ```class_name``` не "зашито" в код ```Vector2d.__repr__```, а получается в виде ```type(self).__name__```. 
```Python
def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
```
Если "зашить" ```class_name```, то подклассы ```Vector2d``` и, в частности, ```ShortVector2d``` должны были бы переопределять метод ```__repr__``` только для того, чтобы изменить ```class_name```. А получая имя от функции ```type```, примененной к экземпляру, ```__repr__``` стал безопасным относительно наследования.